In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'linkedin-freelancer-survey-results:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F1514752%2F2501503%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240422%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240422T191557Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D303716d0a6ec9efa288a39466a56831c1f280f4c259bfd5bdd1de013f03a1c29582e91f3783913388d847501320a993dc9c44399783b386be7ee2bdeeeb66238b9dd2a03f50e526d7d3f7ff549ac2884d229c515dfa16e13ecf482c60d1e22040b837bca8f6e9cf54e05c429c7e160e1b4ec9c2b822fa023a63a8408e997b9305ea66e08a54a3a9d7c7ac557b0920c5b1b706f96cd9880b56c8f9944b6f935ef4a2fa5baf7bcc66b268e610dcd9f6b29fb9ff5686755a4f70954c73bbb5c841c6f71a5abb8bdf99047ff7bc2c4fe5180463612fb11c3cbac663fbfdd0fa2ec2df09b174e4c37e4ac32371f30e661b5c15e0e2703681bb9f5f419a8e6e05a1a80'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


# **Freelancer Survey Analysis - From Clients to Coconuts**

Welcome! This notebook contains the results and insights from 50 polls taken on LinkedIn, which were aimed at freelancers.

The results were gathered by [Dr. Mikko J. Rissanen](https://www.linkedin.com/in/mikkor/), a freelancer working primarily in the AR/VR space. The polls were taken to help give him an insight into the average freelancer, enabling him to produce higher quality content on his website [CoachLancer](https://www.coachlancer.com/). CoachLancer aims to assist anyone on the freelancer journey, with advice ranging from getting more clients to general pro tips. You may also know Mikko as 'that coconut guy from LinkedIn'.

As a freelance data scientist, I thought it would be cool to build upon Mikko's work, compiling the survey results here on Kaggle where others can dissect the data as they see fit. I also thought that it may be helpful to visualize the data so that anyone interested can quickly analyze the data and find their own insights. This data gives a window into the life of a freelancer. As a result, it is primarily aimed at anyone currently working as a freelancer or interested in becoming one. If you're not in that boat but would still like to dig into the findings, please feel free to continue! If you'd like to discuss my conclusions further or offer any suggestions/insights of your own, please feel free to drop me a message [here](http://https://www.linkedin.com/in/alfie-grace-b285ab161/) on LinkedIn. Otherwise, you can drop a comment right here on Kaggle.

*Mikko has provided some of his own insights to contribute to the article. You can recognize Mikko's contributions by looking for any blocks of italicized text, similar to this one!*

**The questions have been split into 6 core themes:**

* The Clients - Choosing a Boss
* The Journey - From Beginner to Veteran
* The Marketing - Generating Leads for a Freelance Business
* The Execution - From Leads to Happy Clients
* The Finances - Picking the Right Price
* The Fun Stuff - Coconuts, Lifestyle and Everything Else

In [ ]:
import math
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt

# load the data
df = pd.read_csv('../input/linkedin-freelancer-survey-results/results.csv')

# create custom functions for generating the visualizations
def dual_pies(colors, t1, r1, o1, t2, r2, o2):
    """ function for generating side-by-side pie charts
    colors - color scheme for the charts
    t1 - title for the first image
    r1 - results for the first image
    o1 - options for the first image
    t2 - title for the second pie
    r2 - results for the second pie
    o2 = options for the second pie """

    # create subplot grid
    fig, (ax1, ax2) = plt.subplots(1,2,figsize=(15,10))

    # create chart 1 and show
    ax1.pie(r1, labels = o1, autopct='%1.0f%%', textprops={'fontsize': 16}, colors=colors)
    ax1.set_title(t1, fontsize=18)

    # create chart 2 and show
    ax2.pie(r2, labels = o2, autopct='%1.0f%%', textprops={'fontsize': 16}, colors=colors)
    dv = ax2.set_title(t2, fontsize=18)

def dual_bars(colors, t1, r1, o1, t2, r2, o2):
    """ function for generating side-by-side bar charts
    variables are the same as in dual_pies """

    # create subplot grid
    fig, (bp1, bp2) = plt.subplots(1, 2, figsize=(15,10))

    # create chart 1 and show
    bp = sns.barplot(ax=bp1, x=o1, y=r1, palette=colors)
    bp.set_ylabel('Respondents', fontsize=14)
    bp.tick_params(labelsize=14)
    bp1.set_title(t1, fontsize=18)

    # create chart 2 and show
    bp = sns.barplot(ax=bp2, x=o2, y=r2, palette=colors)
    bp.set_ylabel('Respondents', fontsize=14)
    bp.tick_params(labelsize=14)
    dv = bp2.set_title(t2, fontsize=18)

def horizontal_bar(colors, t1, r1, o1):
    """ function for generating a single sideways bar chart
    variables are the same as in previous functions """

    # create chart and show
    plt.figure(figsize=(15, 5))
    plt.title(t1, fontsize=18)
    bp = sns.barplot(x=r1, y=o1, palette=colors)
    bp.set_xlabel('Respondents', fontsize=14)
    bp.tick_params(labelsize=14)
    plt.show()

# disable warnings
import warnings
warnings.filterwarnings("ignore")

## **The Clients - Choosing a Boss**

Arguably, the most essential part of freelancing is the clients. It doesn't matter how good your skills are without someone who will pay you to use them. This section will cover clients, aiming to assess freelancer views on client numbers and client interaction.

In [ ]:
# Questions 46 and 37/19

# pick colors for the graphs
colors = ['#7fffd4', '#ffccbb', '#afe79d', '#fff9c4']

# define data Q46
t1 = 'How many clients do you serve in parallel as a freelancer?\n(146 respondents)'
data = df[df.question == 46].iloc[0]
o1 = ['Just one,\nfull-time', '2-3', '4-5', '6 or more']
r1 = [data.A, data.B, data.C, data.D]

# define data Q37/19 (these two are very similar, so the results are combined)
t2 = 'What type of clients do you work with?\n(115 respondents)'
o2 = ['Individuals', 'Startups', 'Medium-size companies', 'Large corporations']
r2 = [0.34, 0.30, 0.22, 0.14]

# create the charts
dual_pies(colors, t1, r1, o1, t2, r2, o2)

It makes sense that an overwhelming majority of the respondents have selected 2 to 3 clients in parallel as their preferred working style. Freelancing can be a risky business, but you reduce the damage that losing a single client can have on your finances by diversifying your income sources. Unfortunately, contracts reach a natural end all the time. With multiple clients part-time, you only need to replace the income from one client when this happens, resulting in far fewer sleepless nights!

On the other hand, it's surprising that 14% of the respondents work with 6 clients or more. The most amount of clients I've ever worked with concurrently is four and I found that a lot of juggling was involved to ensure I was keeping them all happy. So 6 seems like it would definitely be a struggle, although I suppose it also depends on what kind of services you're offering.

Don't quote me on this, but the 34/30/22/14 split on client types could be related to the experience distribution of the freelancers surveyed. Generally speaking, only the most experienced freelancers have access to those big-ticket corporate clients that pay by the bucketload, whereas everyone has access to individual clients.

In [ ]:
# Questions 13 and 48

# define data Q13
t1 = 'How often do you have a disagreement with\na client about the price of your work?\n(32 respondents)'
data = df[df.question == 13].iloc[0]
data = data[1:5].apply(lambda col: round(col*data.respondents, 0))
o1 = ['Never', 'Almost every\ntime', "No, it's\ncalled\n'negotiation'", 'Sometimes']
r1 = [data.A, data.C, data.D, data.B]

# define data Q48
t2 = 'Have you ever\nfired a client?\n(148 respondents)'
data = df[df.question == 48].iloc[0]
data = data[1:5].apply(lambda col: round(col*data.respondents, 0))
o2 = ['Yes, many\ntimes', 'No, never', 'Yes, just once', "What?\nFreelancers\ndon't do that"]
r2 = [data.B, data.C, data.A, data.D]

# create the charts
dual_bars('pastel', t1, r1, o1, t2, r2, o2)

I'm shocked to see that approximately 35% of respondents have never fired a client. Personally, I fall into the 'Yes, just once' column on this one and I believe it isn't a decision to be taken lightly, but sometimes a client just has to go! Perhaps some freelancers are just luckier than others with their clients?

*Firing freelancers is a sadly common phenomenon, but it works the other way round also. 20% of the freelancers report having FIRED A CLIENT once. 37% say they’ve done it several times! Way to go, guys!*

In [ ]:
# Question 44

# define data
t1 = 'How happy are you with the number of new clients you get every month?\n(135 respondents)'
data = df[df.question == 44].iloc[0]
data = data[1:5].apply(lambda col: round(col*data.respondents, 0))
o1 = ['I have a perfect balance', "I only work with long-term clients", "Too happy, I'm overbooked",
           "Not very, I need more clients!"]
r1 = [data.B, data.D, data.C, data.A]

# create chart and show
horizontal_bar('pastel', t1, r1, o1)

The majority here is in the 'Not very, I need more clients!' boat. One of the main joys of freelancing is working with new people on fun new projects, so it makes sense that even well-established freelancers are already looking for the next client. Personally, it's been three months since I've taken on a new client, so maybe it's time I put myself back out there.

In [ ]:
# Questions 8 and 22

# pick colors for the graphs
colors = ['#7fffd4', '#ffccbb', '#afe79d', '#fff9c4']

# define data Q8
t1 = 'Do your clients have the same skillset\nas yourself?\n(23 respondents)'
data = df[df.question == 8].iloc[0]
o1 = [' Yes, all\nof them', 'Yes, some of them', 'No, none of them']
r1 = [data.A, data.B, data.C]

# define data Q22
t2 = 'Do you know how your typical clients make money,\nwhat type of business do they do?\n(46 respondents)'
data = df[df.question == 22].iloc[0]
o2 = ['B2B', 'B2C', 'Business-to-Coconuts (B2CC)', "Don't know, don't care!"]
r2 = [data.A, data.B, data.C, data.D]

# create the charts
dual_pies(colors, t1, r1, o1, t2, r2, o2)

I'm glad to see that it's just a minority falling into the 'Don't know, don't care!' category. The reality is, when talking about your contract history with potential clients, it can be beneficial to understand whether your ex-clients make money business-to-business (B2B) or business-to-customer (B2C). Unfortunately, I haven't worked with anyone on the business-to-coconuts model just yet, so I can't quite comment on that one.

In [ ]:
# Question 15

# define data
t1 = 'What do you mainly look for when you are about to work with a new client?\n(36 respondents)'
data = df[df.question == 15].iloc[0]
data = data[1:5].apply(lambda col: round(col*data.respondents, 0))
o1 = ["Fun collaboration", "Challenging work", 'Money', "Long-term relationship"]
r1 = [data.B, data.D, data.A, data.C]

# create chart and show
horizontal_bar('pastel', t1, r1, o1)

*Hmm… would it not be too much like having a boss after all? I personally get exhausted in long-term projects if they start to repeat anything I could call a routine. I learn nothing new from routines.*

In [ ]:
# pick colors for the graphs
colors = ['#7fffd4', '#ffccbb', '#afe79d', '#fff9c4']

# define data Q5
t1 = 'What method do you use for screening\nyour clients on a freelance site\n(e.g. Upwork, PeoplePerHour, etc.)\n(17 respondents)'
data = df[df.question == 5].iloc[0]
o1 = ['Text messaging', 'One audio call', 'One video call', "   Multiple\n   in-depth\ndiscussions"]
r1 = [data.A, data.B, data.C, data.D]

# define data Q18
t2 = 'Which type of business do\nyou think is better\nfor freelancing\n(41 respondents)'
data = df[df.question == 18].iloc[0]
o2 = ['Be a small fish\nin a big pond ', '  Be a big fish\nin a small pond']
r2 = [data.A, data.B]

# create the charts
dual_pies(colors, t1, r1, o1, t2, r2, o2)

*I would encourage people to be the BIG FISH in whatever pond they choose. Be the Go-To who nearly owns a niche or two, particularly, if you do things online. The world is big enough for a one-person business to work on a very specific niche. Also, that is by far the best way to start on freelance sites.*

## **The Journey - From Beginner to Veteran**

Every business needs a plan to be successful; this includes freelance businesses too! One particular area which should be near the forefront of every freelancer's business plan is growth. If you're not landing those big-ticket clients just yet, that's ok, but you should have a roadmap for how to get there. Alternatively, maybe you're already one of those big fish that Mikko mentioned in the previous section, but if you'd like to stay on top, you still need a growth plan to keep on top of the latest technologies. So whether you're a beginner wondering how to get started in the world of freelancing or if you're someone more experienced that wants to get to the next level, then you may find this section pretty helpful.

In [ ]:
# Question 28

# define data
t1 = 'What is the best way to start freelancing?\n(59 respondents)'
data = df[df.question == 28].iloc[0]
data = data[1:5].apply(lambda col: round(col*data.respondents, 0))
o1 = ["Freelance for previous employers", "Join professional groups", "Use personal connections", 'Join a freelance site']
r1 = [data.A, data.D, data.B, data.C]

# create chart and show
horizontal_bar('vlag', t1, r1, o1)

*46% believe that the best way to start #freelancing is to join a freelance site. Think again! For instance, half of the people joining Upwork never see a penny!*

In [ ]:
# Questions 23 and 49

# define data Q23
t1 = 'Which area of your freelance business are\nyou planning to improve in 2021?\n(49 respondents)'
data = df[df.question == 23].iloc[0]
data = data[1:5].apply(lambda col: round(col*data.respondents, 0))
o1 = ['More clients', 'Better clients', "Better work\nprocess", 'Less work,\nmore fun']
r1 = [data.A, data.B, data.C, data.D]

# define data Q49
t2 = 'How hard do you think it is to build a good\nfreelance business on Upwork?\n(302 respondents)'
data = df[df.question == 49].iloc[0]
data = data[1:5].apply(lambda col: round(col*data.respondents, 0))
o2 = ['Impossible,\nalmost nobody\ncan do it', 'Not at\nall, anyone\ncan do it', 'Not that\nhard, most\ncan do it', "Very hard,\nmost can't\ndo it"]
r2 = [data.A, data.D, data.C, data.B]

# create the charts
dual_bars('vlag', t1, r1, o1, t2, r2, o2)

It seems that the main priority for most respondents is to get more clients; for the most part, more clients = more money, so it makes sense this is something that a lot of freelancers are focused on.

Building a freelance business on Upwork can be challenging, but it's definitely doable. If it was easy, then everyone would do it!

*51% of the freelancers think Upwork is either very or insanely hard for building a freelance business. No surprise there. That place is tough.*

In [ ]:
# Questions 5 and 7

# pick colors for the graphs
colors = ['#6e90bf', '#aab8d0', '#e4e5eb', '#f2dfdd']

# define data Q5
t1 = 'How long would you say you\nstruggled before your freelance business\nbrought you satisfactory income?\n(19 respondents)'
data = df[df.question == 6].iloc[0]
o1 = ['Weeks', 'Months', 'Half a year', "One year or more"]
r1 = [data.A, data.B, data.C, data.D]

# define data Q7
t2 = 'Have you ever pivoted\nyour freelance business from\none service to another?\n(20 respondents)'
data = df[df.question == 7].iloc[0]
o2 = ['No, not yet', 'No, I never will', 'Yes, once', 'Yes, many times']
r2 = [data.A, data.B, data.C, data.D]

# create the charts
dual_pies(colors, t1, r1, o1, t2, r2, o2)

Freelancing is the same as most things in life because what you get out of it is equivalent to what you put in. As a result, the amount of time it takes to start bringing in an adequate income is directly related to how much hard work you're willing to put into building your reputation and your portfolio.

*I think change is good just for keeping things interesting, if for no other reason.*

In [ ]:
# Questions 26 and 35

# define data Q26
t1 = 'Do you have a long-term strategy that\nyou follow when considering which\nfreelance jobs to start working on?\n(53 respondents)'
data = df[df.question == 26].iloc[0]
data = data[1:5].apply(lambda col: round(col*data.respondents, 0))
o1 = ['No,\nstrategy is\nunnecessary', 'Yes, for\ngrowing\na team', "Yes, for\ngrowing\nclientele", 'Yes, for\nexpanding\nmy\nskill set']
r1 = [data.A, data.D, data.C, data.B]

# define data Q35
t2 = 'Which of these critical areas\nof freelancing would you like\nto learn more about?\n(68 respondents)'
data = df[df.question == 35].iloc[0]
data = data[1:5].apply(lambda col: round(col*data.respondents, 0))
o2 = ["Branding,\nmarketing\nand sales", 'Client\nhandling and\ncommunication', 'Strategic\nniche\nbuilding', 'Coconut\njuggling']
r2 = [data.A, data.B, data.C, data.D]

# create the charts
dual_bars('vlag', t1, r1, o1, t2, r2, o2)

*Not having a strategy is close to business suicide. How else would you plan to stay competitive as the world changes constantly?*

Most respondents are looking to learn more about branding, marketing and sales. This desire is understandable, you need to build your skills to earn higher rates, but you also need the marketing know-how to advertise those skills to clients.

In [ ]:
# Question 50

# define data
t1 = 'How long have you been freelancing?\n(478 respondents)'
data = df[df.question == 50].iloc[0]
data = data[1:5].apply(lambda col: round(col*data.respondents, 0))
o1 = ["More than 10 (I'm a veteran)!", "6-10 years", "1-5 years", "Less than 1 year"]
r1 = [data.D, data.C, data.B, data.A]

# create chart and show
horizontal_bar('vlag', t1, r1, o1)

It's no surprise at all to see such a considerable portion of new freelancers. It's no secret that the high unemployment rates caused by COVID-19 have resulted in a massive influx of new freelancers eager to try out a new way of working.

In [ ]:
# Questions 42 and 47

# pick colors for the graphs
colors = ['#6e90bf', '#aab8d0', '#e4e5eb', '#f2dfdd']

# define data Q42
t1 = 'How many more years are\nyou planning to freelance?\n(101 respondents)'
data = df[df.question == 42].iloc[0]
o1 = [" None, I'm\nsoon done!", "1-5 years", "6-10 years", "Forever! (freelancer for life)"]
r1 = [data.A, data.B, data.C, data.D]

# define data Q47
t2 = 'Which world is better for\nlong-term freelance success?\n(147 respondents)'
data = df[df.question == 47].iloc[0]
o2 = ["Local offline world", "Global online world", "A mixture of both"]
r2 = [data.A, data.B, data.C]

# create the charts
dual_pies(colors, t1, r1, o1, t2, r2, o2)

*57% plan to freelance for life! Supposedly, they are those who have got things going very sweet, are perhaps based out of an island of #coconuts, sipping drinks on a beach while working, etc. That is a VERY NICE PERCENTAGE!*

Building upon my previous point about the effect that COVID-19 has had on freelancers, it's also no shock to see that most respondents believe it's better to build a freelance business online. The pandemic has shown employers that you don't need someone to be physically sat in the same building to get results. For freelancers, this is excellent news as it opens up a global marketplace of opportunities.

## **The Marketing - Generating Leads for a Freelance Business**

You could be the most talented data analyst, graphic designer or copywriter out there, but it doesn't matter if you can't land a gig. Marketing is one of the most critical areas of freelancing, so read on to see what some of our respondents had to say about how they go about hunting for leads.

In [ ]:
# define data
t1 = 'What is your favourite freelance site for your freelance business?\n(95 respondents)'
data = df[df.question == 41].iloc[0]
data = data[1:5].apply(lambda col: round(col*data.respondents, 0))
o1 = ["freelancer.com", "fiverr.com", "None of the above", "upwork.com"]
r1 = [data.B, data.A, data.D, data.C]

# create chart and show
horizontal_bar('coolwarm', t1, r1, o1)

The impression I've gotten from various sources is that different freelance sites can be suitable for different types of services. That being said, I can see why Upwork is doing so well. They've been killing it recently!

In [ ]:
# Questions 24 and 34

# pick colors for the graphs
colors = ['#6788ee', '#c9d7f0', '#edd1c2', '#f7a889']

# define data Q24
t1 = 'How much do you trust\nyour favourite freelance site?\n(50 respondents)'
data = df[df.question == 24].iloc[0]
o1 = ["Not at all, it\n could turn\n really bad","A lot, I'm safe", "\nNot sure, it could\n  go either way", "Don't care,\n I've got a\n   plan B"]
r1 = [data.A, data.B, data.C, data.D]

# define data Q34
t2 = 'Would you join good old Elance, if it was\nresurrected and not replacing the current Upwork?\n(66 respondents)'
data = df[df.question == 34].iloc[0]
o2 = ["Yes", "No", "What was     \nElance anyway?", "        Don't care,\nmore coconuts please"]
r2 = [data.A, data.B, data.C, data.D]

# create the charts
dual_pies(colors, t1, r1, o1, t2, r2, o2)

I'm stunned to see that a whopping 50% of respondents have complete trust in their favorite freelance site; it takes a lot to have full confidence that your primary source of income won't hang you out to dry under certain circumstances. The general consensus that I've seen is that it's best to diversify your income sources. That way, if one website cuts you off for whatever reason, you've got a backup plan in place.

In [ ]:
# Questions 43 and 45

# define data Q45
t1 = 'How happy are you with your online presence?\n(LinkedIn, social media, freelance sites, etc.)\n(142 respondents)'
data = df[df.question == 45].iloc[0]
data = data[1:5].apply(lambda col: round(col*data.respondents, 0))
o1 = ["I'm overloaded\nwith\nopportunities!", "Not at\nall, I'm\ninvisible", "I'm just\ngetting started", "I'm doing\njust fine"]
r1 = [data.D, data.A, data.B, data.C]

# define data Q43
t2 = 'What is the best hashtag\nfor freelancers on LinkedIn?\n(125 respondents)'
data = df[df.question == 43].iloc[0]
data = data[1:5].apply(lambda col: round(col*data.respondents, 0))
o2 = ["#upwork", "#freelance", "#freelancing", "#coconuts"]
r2 = [data.C, data.A, data.B, data.D]

# create the charts
dual_bars('coolwarm', t1, r1, o1, t2, r2, o2)

Sorry Mikko, it looks like #coconuts hasn't taken off just yet!

In [ ]:
# Questions 3 and 17

# pick colors for the graphs
colors = ['#6788ee', '#c9d7f0', '#edd1c2', '#f7a889']

# define data Q3
t1 = 'Which types of professional\norganizations do you belong to for finding\nclients for your freelance business?\n(13 respondents)'
data = df[df.question == 3].iloc[0]
o1 = ["Freelancers' union\n       or similar", "Paid          \nmembership    \norganization    ", "Free membership\norganization    ", "None of the above"]
r1 = [data.A, data.B, data.C, data.D]

# define data Q17
t2 = 'Do you sell your\nfreelance services through an\nkind of sales partner?\n(37 respondents)'
data = df[df.question == 17].iloc[0]
o2 = ["No, never", "Yes, sometimes I use partners", "Yes, I use\n partners\n  always"]
r2 = [data.A, data.B, data.C]

# create the charts
dual_pies(colors, t1, r1, o1, t2, r2, o2)

Personally, I've never had to join any organizations or work with sales partners to help get more business. That being said, I know that many people struggle with finding clients at first, so I can see why they'd perhaps try a different approach at times.

*I find it very interesting that some manage their sales without interfacing with the client directly!*

## **The Execution - From Leads to Happy Clients**

This section is dedicated to the day-to-day of freelancer life, looking at some of their key challenges and how freelancers like to work.

In [ ]:
# Questions 33 and 16

# define data Q33
t1 = 'What is the hardest part\nof freelancing?\n(66 respondents)'
data = df[df.question == 33].iloc[0]
data = data[1:5].apply(lambda col: round(col*data.respondents, 0))
o1 = ["Getting\nclients\nto pay well", "Converting leads\nto clients", "Finding\nleads", "Finishing the\nwork on time"]
r1 = [data.C, data.B, data.A, data.D]

# define data Q16
t2 = "What type of problems did you experience in\nthe hardest freelance project you've ever done?\n(33 respondents)"
data = df[df.question == 16].iloc[0]
data = data[1:5].apply(lambda col: round(col*data.respondents, 0))
o2 = ["Time\nmanagement", "Communicating\nwith\nthe client", "Work\nprocess and\nexecution", "Changing\nrequirements"]
r2 = [data.A, data.C, data.D, data.B]

# create the charts
dual_bars('pastel', t1, r1, o1, t2, r2, o2)


Interestingly, only a slim majority of respondents thank that actually doing the work is the easy part; maybe looking for more challenging gigs will result in clients that pay better!

Changing requirements is definitely a struggle and it's easy to see why a lot of the freelancers surveyed would agree. There's not much that's more demoralizing than when you're ready to wrap up on a project only for the client to change their expectations at the last minute.

In [ ]:
# Questions 12 and 27

# pick colors for the graphs
colors = ['#7fffd4', '#ffccbb', '#afe79d', '#fff9c4']

# define data Q12
t1 = 'Do you have any collaborators as\npart of your freelance business?\n(31 respondents)'
data = df[df.question == 12].iloc[0]
o1 = ["   I have\nmarketing\n     help", "I have sales\n  partners", "No, I don't need anyone", "I have       \nsubcontractors"]
r1 = [data.C, data.D, data.A, data.B]

# define data Q27
t2 = 'As a freelancer, how often do\nyou hire other freelancers?\n(55 respondents)'
data = df[df.question == 27].iloc[0]
o2 = ["Never", "\n\nSometimes", "Regularly", "All the time"]
r2 = [data.A, data.B, data.C, data.D]

# create the charts
dual_pies(colors, t1, r1, o1, t2, r2, o2)

Likely, these results are highly correlated to the types of jobs that freelancers do. Some gigs just don't require more than one pair of hands to take care of, so, understandably, many freelancers don't require collaborators or subcontractors.

In [ ]:
# define data
t1 = "What kind of impact do you have on your client's business?\n(44 respondents)"
data = df[df.question == 21].iloc[0]
data = data[1:5].apply(lambda col: round(col*data.respondents, 0))
o1 = ["Don't know, no access to data", "Make a critical part of the business","Save more money", "Make more money"]
r1 = [data.D, data.C, data.B, data.A]

# create chart and show
horizontal_bar('pastel', t1, r1, o1)

In [ ]:
# Questions 32 and 40

# define data Q32
t1 = 'Which type of freelance contracts do you prefer?\n(65 respondents)'
data = df[df.question == 32].iloc[0]
data = data[1:5].apply(lambda col: round(col*data.respondents, 0))
o1 = ["Short-term,\nfull-time", "Long-term,\nfull-time", "Short-term,\npart-time", "Long-term,\npart-time"]
r1 = [data.B, data.D, data.A, data.C]

# define data Q40
t2 = "What is your preferred method of freelancing?\n(86 respondents)"
data = df[df.question == 40].iloc[0]
data = data[1:5].apply(lambda col: round(col*data.respondents, 0))
o2 = ["Fully online\n(e.g. LinkedIn\nand Upwork)", "Mostly\nonline", "Mostly\noffline", "Fully offline\n(location-based)"]
r2 = [data.D, data.C, data.B, data.A]

# create the charts
dual_bars('pastel', t1, r1, o1, t2, r2, o2)

From a financial standpoint, it's understandable that 'Long-term, part-time' is the winner here. With 2 or 3 long-term/part-time contracts under your belt, you've got a steady source of income for the foreseeable future.

## **The Finances - Picking the Right Price**

Clients, marketing and strategy all lead towards one thing - making money! As a result, it's essential to ensure that your freelance business is geared towards making all the hard work worth your while. This section explores how respondents go about pulling in the correct prices for their services.

In [ ]:
# define data
t1 = "How do you set a price for your freelance projects?\n(63 respondents)"
data = df[df.question == 31].iloc[0]
data = data[1:5].apply(lambda col: round(col*data.respondents, 0))
o1 = ["By looking at competitors", "By effort/hours estimates", "By value to the client", "By predefined package price"]
r1 = [data.A, data.D, data.C, data.B]

# create chart and show
horizontal_bar('Spectral', t1, r1, o1)

It's hard to put an exact price on your services. Theoretically, clients should be paying for the value that a freelancer adds to their business. However, in reality, freelancing can be very competitive, so you also need to be aware of the going rate for your services, especially when you haven't got the pedigree to demand those coveted top-tier rates.

In [ ]:
# Questions 30 and 36

# define data Q30
t1 = 'Which type of freelance\ncontracts do you prefer?\n(62 respondents)'
data = df[df.question == 30].iloc[0]
data = data[1:5].apply(lambda col: round(col*data.respondents, 0))
o1 = ["A nice mix of both", "Hourly", "Fixed-price"]
r1 = [data.C, data.A, data.B]

# define data Q36
t2 = "Do you charge an advance fee\nfor your freelance projects?\n(69 respondents)"
data = df[df.question == 36].iloc[0]
data = data[1:5].apply(lambda col: round(col*data.respondents, 0))
o2 = ["No", "Yes, 0-25%", "Yes, 25-50%", "Yes, 51%\nor more"]
r2 = [data.D, data.C, data.B, data.A]

# create the charts
dual_bars('Spectral', t1, r1, o1, t2, r2, o2)

I can see why most respondents would favor a mixture of both contract types. Fixed-price contracts are outstanding if you can do them quickly, resulting in a nice lump sum and putting a dent in your targets. On the other hand, if you don't plan your time well, fixed-price gigs can very quickly result in you not getting much bang for your buck. The great thing about hourly contracts is that regardless of how long a project takes you, you're going to be reimbursed for your time. However, some freelancers have a problem with hourly gigs because as you become more skilled, you can also get jobs done quicker. This results in less billing time.

In [ ]:
# Questions 6 and 14

# pick colors for the graphs
colors = ['#fca55d', '#fee999', '#edf8a3', '#47a0b3']

# define data Q6
t1 = 'How long would you say you struggled\nbefore your freelance business\nbrought you satisfactory income?\n(19 respondents)'
data = df[df.question == 6].iloc[0]
o1 = ["Weeks", "Months", "Half a year", "One year or more"]
r1 = [data.A, data.B, data.C, data.D]

# define data Q14
t2 = 'How much financial runway (savings\nfor covering your living expenses)\ndid you have when you started freelancing?\n(34 respondents)'
data = df[df.question == 14].iloc[0]
o2 = ["0 months, I felt lucky", "<4 months", "4-6 months    \nas recommended", ">6 months"]
r2 = [data.A, data.B, data.C, data.D]

# create the charts
dual_pies(colors, t1, r1, o1, t2, r2, o2)

*An alerting number of 35% of freelancers didn’t have ANY SAVINGS to cover their living expenses when they started #freelancing! But fortunately, 47% say they had savings enough for the recommended period of at least 4-6 months or even more. Dudes, EVERY BUSINESS needs an initial capital. In the case of freelance businesses, it can be very small, though.*

## **The Fun Stuff - Coconuts, Lifestyle and Everything Else**

In [ ]:
# Questions 25 and 29

# pick colors for the graphs
colors = ['#ae8772', '#a9d89a', '#f9fb1d', '#d1f9f0']

# define data Q25
t1 = 'How do you see your coconut\nonce 50% of it has been consumed?\n(53 respondents)'
data = df[df.question == 25].iloc[0]
o1 = ["It's half\n empty", "It's half full", "Finish it, then get another", "\nDon't care,\n   it's just\n    a fruit"]
r1 = [data.A, data.B, data.C, data.D]

# define data Q29
t2 = 'Freelancer, what is\nyour favourite color?\n(60 respondents)'
data = df[df.question == 29].iloc[0]
o2 = ["Coconut brown", "Coconut green", "Coconut yellow", "Blue!"]
r2 = [data.A, data.B, data.C, data.D]

# create the charts
dual_pies(colors, t1, r1, o1, t2, r2, o2)

*When asked about their freelancing philosophy, 28% of freelancers say their COCONUT IS HALF FULL. 0% think it is half empty (as nobody should think). Fortunately, though, 62% prefer to finish the coconut, then get another, which is the right way to freelance. Sadly, 10% don’t give a ****, because it’s just a fruit (but at least now they know coconut is technically not a nut at all, it is indeed a fruit).*

*42% of freelancers say their FAVORITE COLOR IS COCONUT GREEN. Coconut brown came second (20%), coconut yellow third (7%). Blue was the wrong answer, even if 31% thought the opposite. Obviously, it’s just wrong, no matter what Monty Python says. Needless to mention any other colors as they are simply not well-suited for freelancers*

In [ ]:
# Question 39

# define data
t1 = 'How do you expect COVID-19 to affect your freelance business in 2021?\n(77 respondents)'
data = df[df.question == 39].iloc[0]
data = data[1:5].apply(lambda col: round(col*data.respondents, 0))
o1 = ["Less competition", "Less opportunities", "More competition", "More opportunities"]
r1 = [data.C, data.A, data.D, data.B]

# create chart and show
horizontal_bar('Spectral', t1, r1, o1)

More competition and more opportunities seems to be the general consensus here. As mentioned previously, the high unemployment rates caused by COVID-19 have resulted in many people trying their hand at freelancing. Moreover, due to the tough economy, the increase in competition has also been matched by more companies outsourcing as a cheaper alternative to hiring someone full-time.

In [ ]:
# Questions 9 and 38

# pick colors for the graphs
colors = ['#fca55d', '#fee999', '#edf8a3', '#47a0b3']

# define data Q9
t1 = 'Freelancing is great, but are there any\nparts that you miss from your old day job?\n(23 respondents)'
data = df[df.question == 9].iloc[0]
o1 = ["Great work community", "Stable salary", "Risk-free work life", "  Clarity in\n      work\nassignments"]
r1 = [data.A, data.B, data.C, data.D]

# define data Q38
t2 = 'Freelancers, what is your\nmain reason for freelancing?\n(74 respondents)'
data = df[df.question == 38].iloc[0]
o2 = ["Better money", "Can work from home", "\nFreeeee\neeeeee\ndooom!", "Coconuts"]
r2 = [data.A, data.B, data.C, data.D]

# create the charts
dual_pies(colors, t1, r1, o1, t2, r2, o2)

*Only 9% say it is coconuts. This result is highly disappointing to me, personally. Don’t all freelancers want true success?*

In [ ]:
# Questions 11 and 20

# define data Q11
t1 = 'Freelancer, how much time off do you\nmanage to take during Christmas break?\n(29 respondents)'
data = df[df.question == 11].iloc[0]
data = data[1:5].apply(lambda col: round(col*data.respondents, 0))
o1 = ["What Christmas\nbreak?", "More than\n4 days", "1-2 days", "4 days"]
r1 = [data.D, data.A, data.C, data.B]

# define data Q20
t2 = 'If you could make one wish to change\nsomething about online freelancing, what would it be?\n(44 respondents)'
data = df[df.question == 20].iloc[0]
data = data[1:5].apply(lambda col: round(col*data.respondents, 0))
o2 = ["Better\nfreelance\nsites", "More\nteamwork", "Higher\npay in\ngeneral", "More\nbusiness\nstability"]
r2 = [data.B, data.C, data.A, data.D]

# create the charts
dual_bars('Spectral', t1, r1, o1, t2, r2, o2)

*If 52% of freelancers wish they had a more stable salary and 55% hope to have better business stability, perhaps it is about a quarter of all freelancers who don't seem to be crushing it (yet, at least). It might be a better move to drop the desired for complete "freeeeeedooooom!" and go back to the day job. That would fulfill both of the hopes, wouldn't it? Freelancing is not for everyone.*

I can agree that freelancing isn't for everyone, it's a lot of work and you need to be good at motivating yourself. As a freelancer, there's no boss guiding you to make sure you're performing at your best; something which some people need. However, one of the best things about freelancing is that even though it's hard work, you're working hard for yourself. When you couple this with doing something you love, it can be easy to feel like you aren't doing any work at all. Despite this, it's essential to look after yourself freelancers, so if you're in the 'What Chrismas break?' category, then take that holiday time you've been thinking about!